## Settings

In [26]:
from typing import List

def cdec(module_name):
    def dec_inner(targ_class):
        targ_class.MODULE_NAME: str = module_name

        @property
        def tprop(self):
            return self.a + 100
        targ_class.tprop = tprop

        def inner_init(self, a: int):
            self.a = a
        targ_class.__init__ = inner_init
        
        return targ_class
    return dec_inner

def subcdec(module_name):
    def dec_inner(targ_class):
        # targ_class = cdec(module_name)(targ_class)

        def sub_inner_init(self, b: int):
            self.b = b
            type(self).mro()[1].__init__(self, a=b//2)
        targ_class.__init__ = sub_inner_init
        
        return targ_class
    return dec_inner

@cdec('mod')
class cex:
    __slots__: List[str] = ['a']
    TAG: str='CEX'

    @classmethod
    def get_cls_tag(cls):
        return cls.TAG

    @staticmethod
    def double(x):
        return x*2

@subcdec('modd')
class subcex(cex):
    __slots__: List[str] = ['b']
    TAG: str='SUBCEX'

A = cex(55)
B = subcex(66)

print((A.get_cls_tag(), B.get_cls_tag()))
print(((A.a,), A.tprop, A.MODULE_NAME, id(A.MODULE_NAME), cex.MODULE_NAME, id(cex.MODULE_NAME)))
print(((B.a, B.b), B.tprop, B.MODULE_NAME, id(B.MODULE_NAME), cex.MODULE_NAME, id(cex.MODULE_NAME)))
cex.MODULE_NAME = 'x'
print((A.a, A.tprop, A.MODULE_NAME, id(A.MODULE_NAME), cex.MODULE_NAME, id(cex.MODULE_NAME)))
print((B.a, B.tprop, B.MODULE_NAME, id(B.MODULE_NAME), cex.MODULE_NAME, id(cex.MODULE_NAME)))
print((A.__slots__,B.__slots__))

8
('CEX', 'SUBCEX')
((55,), 155, 'mod', 4379173360, 'mod', 4379173360)
((33, 66), 133, 'mod', 4379173360, 'mod', 4379173360)
(55, 155, 'x', 4378863536, 'x', 4378863536)
(33, 133, 'x', 4378863536, 'x', 4378863536)
(['a'], ['b'])


In [4]:

import scapy.all as scp
import struct
import numpy as np
import matplotlib.pyplot as plt

from typing import DefaultDict

from IrisBackendv3.data_standards import DataStandards
from IrisBackendv3.data_standards.prebuilt import add_to_standards, watchdog_heartbeat_tvac
from IrisBackendv3.data_standards.logging import logger as DsLogger

from IrisBackendv3.codec.magic import Magic, MAGIC_SIZE
from IrisBackendv3.codec.settings import set_codec_standards

from IrisBackendv3.utils.basic import print_bytearray_hex as printraw

In [5]:
DsLogger.setLevel('CRITICAL')
standards = DataStandards.build_standards()
add_to_standards(standards, watchdog_heartbeat_tvac)
standards.print_overview()

Data Standards Overview: [

	Module[256]::BlockDriver

		Commands:

		Telemetry:

			0.	Channel[0]::BdCycles: uint32

		Events:

	Module[512]::RateGroupDriver

		Commands:

		Telemetry:

		Events:

	Module[768]::ActiveRateGroup-RateGroupLowFreq

		Commands:

		Telemetry:

			0.	Channel[0]::RgMaxTime: uint32

			1.	Channel[1]::RgCycleSlips: uint32

		Events:

			0.	Event[0]::RateGroupStarted[]

			1.	Event[1]::RateGroupCycleSlip[cycle: uint32]

	Module[1024]::ActiveRateGroup-RateGroupMedFreq

		Commands:

		Telemetry:

			0.	Channel[0]::RgMaxTime: uint32

			1.	Channel[1]::RgCycleSlips: uint32

		Events:

			0.	Event[0]::RateGroupStarted[]

			1.	Event[1]::RateGroupCycleSlip[cycle: uint32]

	Module[1280]::ActiveRateGroup-RateGroupHiFreq

		Commands:

		Telemetry:

			0.	Channel[0]::RgMaxTime: uint32

			1.	Channel[1]::RgCycleSlips: uint32

		Events:

			0.	Event[0]::RateGroupStarted[]

			1.	Event[1]::RateGroupCycleSlip[cycle: uint32]

	Module[1536]::CubeRoverTime

		Commands:

		Teleme

In [6]:
from IrisBackendv3.codec.fsw_data_codec import encode
bo = '<'
seq_num = 0x00
def pack(name_m: str, name_c: str, data: bytes) -> bytes:
    global seq_num
    module = standards.modules[name_m]
    cmd = module.commands[name_m+'_'+name_c]

    magic = Magic.COMMAND
    magic_bytes = magic.encode(byte_order=bo)
    opcode = struct.pack(bo+'H', module.ID | cmd.ID)

    seq_num = seq_num + 1
    vlp_len = MAGIC_SIZE + 2 + len(data)
    chk = 0x00
    CPH = struct.pack(bo+'B H B', seq_num, vlp_len, chk)

    out = CPH + magic_bytes + opcode + data
    return out
    
data = struct.pack(bo+'B H', 0x01, 0x0005) # 77 for modes, 60 for prep/deploy
packet = pack('Camera', 'TakeImage', data) 
printraw(packet)
print(packet.hex())

01 09 00 00 55 da ba 00 01 11 01 05 00
0109000055daba000111010500


In [7]:
standards.modules['WatchDogInterface'].commands.vals
sorted([m.ID for m in standards.modules.vals])

[256,
 512,
 768,
 1024,
 1280,
 1536,
 1792,
 2048,
 2304,
 2560,
 2816,
 3328,
 3584,
 3840,
 4096,
 4352,
 65280]

In [8]:
from IrisBackendv3.codec.payload import Payload, CommandPayload, WatchdogCommandPayload
from IrisBackendv3.codec.metadata import DataPathway, DataSource
from IrisBackendv3.codec.magic import Magic
from IrisBackendv3.codec.settings import ENDIANNESS_CODE, set_codec_standards

import struct

set_codec_standards(standards)

def pack_watchdog(command_name: str, **kwargs) -> bytes:
    module, command = standards.global_command_lookup(command_name)
    payload = WatchdogCommandPayload(
        pathway = DataPathway.WIRED,
        source = DataSource.GENERATED,
        magic = Magic.WATCHDOG_COMMAND,
        module_id = module.ID,
        command_id = command.ID,
        args = kwargs
    )
    return payload.encode()
packet = Magic.WATCHDOG_COMMAND.encode() + pack_watchdog('WatchDogInterface_SetVSetpoint', setpoint=1000) # camera_num=0x01, callback_id = 0x05)

vlp_len = len(packet)
checksum = 0 # ignore for now TODO: impl. me
seq_num = 0 # watchdog doesn't care
CPH = struct.pack(ENDIANNESS_CODE + 'B H B', seq_num, vlp_len, checksum)
packet = CPH + packet

import scapy.all as scp
full_packet = scp.IP(dst='127.0.0.1', src='222.173.190.239')/scp.UDP(dport=8080)/scp.Raw(load=packet)
printraw(scp.raw(scp.IP(scp.raw(full_packet))))
printraw(scp.raw(full_packet))
scp.raw(full_packet)
len(full_packet)


45 00 00 28 00 01 00 00 40 11 5e 26 de ad be ef 7f 00 00 01 00 35 1f 90 00 14 11 86 00 08 00 00 ee ff 00 c0 da 10 e8 03
45 00 00 28 00 01 00 00 40 11 5e 26 de ad be ef 7f 00 00 01 00 35 1f 90 00 14 11 86 00 08 00 00 ee ff 00 c0 da 10 e8 03


40

In [9]:
from __future__ import annotations
from typing import Optional, Dict, List
import struct
import pickle
from abc import abstractmethod
from enum import Enum

class Dummy:
    __slots__: List[str] = [
        '_raw',
        '_endianness_code'
    ]

    class FileTypeMagic(Enum):
        """
        Enumeration of all file types which could be downlinked.

        As usual, for backwards compatibility and data preservation, *never* change 
        any of the enum values or delete entries, just deprecate old ones.
        """
        IMAGE = 0x01
        UWB = 0x0F

    FTM: FileTypeMagic = FileTypeMagic.UWB

    _raw: Optional[bytes]
    _endianness_code: str

    def __init__(self,
        raw: Optional[bytes] = None,
        endianness_code = '!'
    ) -> None:
        self._raw = raw
        self._endianness_code = endianness_code

    @abstractmethod
    def encode(self) -> bytes:
        raise NotImplementedError()

    @classmethod
    @abstractmethod
    def decode(cls, raw: bytes, endianness_code:str) -> Dummy:
        raise NotImplementedError()

    def __reduce__(self) -> Tuple[Callable, Tuple[bytes, str], Optional[str]]:
        print("Dummy Reduce")

        if self._raw is None:
            self._raw = self.encode()

        if hasattr(self, '__getstate__'):
            state = self.__getattribute__('__getstate__')()
        else:
            state = None

        # "Callable object" returned will be the decoding function:
        # If a subclassed object is reduced, it will call that subclass' `decode`
        # function (assuming it's been implemented).
        return (self.__class__.decode, (self._raw, self._endianness_code), state)

class DummerInterface(Dummy):
    __slots__: List[str] = [
        'other_thing',
        '_data'
    ]

    other_thing: str
    _data: int

class DummyThicc(DummerInterface):
    __slots__: List[str] = []

    # FTM2: DummyThicc.FileTypeMagic = DummyThicc.FileTypeMagic.IMAGE

    def __init__(self,
        data: int,
        other_thing: str = "",
        raw: Optional[bytes] = None,
        endianness_code = '!'
    ) -> None:
        self.other_thing = other_thing
        self._data = data
        super().__init__(raw, endianness_code)

    def encode(self) -> bytes:
        print("Encoding")
        return struct.pack(self._endianness_code+'L', self._data)

    @classmethod
    def decode(cls, raw: bytes, endianness_code:str) -> Dummy:
        print("Decoding")
        return cls(
            data = struct.unpack(endianness_code+'L', raw)[0],
            raw = raw,
            endianness_code = endianness_code
        )

    def __repr__(self) -> str:
        return f"{self.other_thing}[{self._data}] -> {self._endianness_code}{self._raw}"

    def __eq__(self, other) -> bool:
        return self.other_thing == other.other_thing and self._data == other._data

    def __getstate__(self) -> str:
        print("Getting State")
        return self.other_thing

    def __setstate__(self, state: str) -> None:
        print("Setting State")
        self.other_thing = state


A = DummyThicc(other_thing='Apple', data=5)
B = DummyThicc(other_thing='Dead', data=0xBEEF)
print((A, B))

print('--pickle--')
pA = pickle.dumps(A)
pB = pickle.dumps(B)
print('--unpickle--')
upA = pickle.loads(pA)
upB = pickle.loads(pB)

print((A, B))
print((upA, upB))
print((upA==A, upB==B))

from typing import NamedTuple

class X(NamedTuple):
    a: int
    b: str

x = X(
    a = 0,
    b = '5'
)
getattr(x, 'b')
# Dummy.FTM == DummyThicc.FileTypeMagic.UWB, DummyThicc.FTM2
DummerInterface.__slots__


(Apple[5] -> !None, Dead[48879] -> !None)
--pickle--
Dummy Reduce
Encoding
Getting State
Dummy Reduce
Encoding
Getting State
--unpickle--
Decoding
Setting State
Decoding
Setting State
(Apple[5] -> !b'\x00\x00\x00\x05', Dead[48879] -> !b'\x00\x00\xbe\xef')
(Apple[5] -> !b'\x00\x00\x00\x05', Dead[48879] -> !b'\x00\x00\xbe\xef')
(True, True)


['other_thing', '_data']

In [10]:
# Data Transport:
file = './test-data/Iris_FSWv1.0.0_210409_Telemetry.pcapng' # PCAP logs
protocol = scp.UDP # Protocol FSW is using to send data
port = 8080 # Port on the spacecraft FSW is sending data to

# Data Formatting Settings:
packetgap = 0 # number of packets to ignore at beginning of pcap
deadspace = 0 # number of bytes of deadspace at the beginning of the
endianness_code = "<" # < = little, > = big, ! = network

# Image Formatting Settings:
image_settings = {
    "width": 2592,
    "height": 1944
}

## Decode First Packet
### Grab Packet

In [11]:
pcap = scp.rdpcap(file)

In [12]:
packets = list(filter(lambda x: x.dport == port, pcap[protocol][packetgap:]))
i = 0

In [13]:
packet = packets[i]

In [14]:
content = scp.raw(packet.getlayer(scp.Raw))[deadspace:]
printraw(content)
packet_bytes = content
# packet_bytes

00 cc 03 90 ff 10 00 c0 00 0f 00 00 00 00 04 00 ff 10 00 c0 01 0f 00 00 00 00 fd ff ff 10 00 c0 02 0f 00 00 00 00 9f 00 ff 10 00 c0 03 0f 00 00 00 00 43 00 ff 10 00 c0 04 0f 00 00 00 00 2e 00 ff 10 00 c0 05 0f 00 00 00 00 d3 ff ff 10 00 c0 00 05 00 00 00 00 00 00 00 00 ff 10 00 c0 00 04 02 00 00 00 d0 07 00 00 ff 10 00 c0 00 03 23 00 00 00 b8 88 00 00 ff 10 00 c0 00 0f f2 00 00 00 09 00 ff 10 00 c0 01 0f f2 00 00 00 fd ff ff 10 00 c0 02 0f f2 00 00 00 a0 00 ff 10 00 c0 03 0f f2 00 00 00 3b 00 ff 10 00 c0 04 0f f2 00 00 00 1d 00 ff 10 00 c0 05 0f f2 00 00 00 de ff ff 10 00 c0 00 05 24 00 00 00 e8 03 00 00 ff 10 00 c0 00 0f e5 01 00 00 04 00 ff 10 00 c0 01 0f e5 01 00 00 fd ff ff 10 00 c0 02 0f e5 01 00 00 9c 00 ff 10 00 c0 03 0f e5 01 00 00 3a 00 ff 10 00 c0 04 0f e5 01 00 00 2f 00 ff 10 00 c0 05 0f e5 01 00 00 e4 ff ff 10 00 c0 00 03 16 01 00 00 a0 8c 00 00 ff 10 00 c0 00 0f d7 02 00 00 05 00 ff 10 00 c0 01 0f d7 02 00 00 fe ff ff 10 00 c0 02 0f d7 02 00 00 a0 00 ff 10 00 c0 03 0f d7 0

### Decode Common Packet Header

In [15]:
CPH_SIZE = 4
CPH = packet_bytes[:CPH_SIZE]
printraw(CPH)
cph_head, checksum = CPH[:-1], CPH[-1:]
seq_num, vlp_len = struct.unpack(endianness_code+' B H', cph_head)
print(seq_num, vlp_len, checksum)
#! TODO: Perform checksum check. (not impl. in FSW atm)
assert vlp_len == len(packet_bytes) - CPH_SIZE

00 cc 03 90
0 972 b'\x90'


### Decode Variable Length Payload

In [16]:
from IrisBackendv3.codec.payload import extract_downlinked_payloads
from IrisBackendv3.codec.metadata import DataPathway, DataSource
from IrisBackendv3.codec.magic import Magic

# Extract the Variable Length Payload
VLP = packet_bytes[CPH_SIZE:]
# printraw(VLP)


# Parse VLP:
payloads = extract_downlinked_payloads(
    VLP = VLP,
    pathway = DataPathway.WIRELESS,
    source = DataSource.PCAP
)

In [17]:
sorted(standards.modules.vals, key = lambda m: m.ID)

[Module[256]::BlockDriver,
 Module[512]::RateGroupDriver,
 Module[768]::ActiveRateGroup-RateGroupLowFreq,
 Module[1024]::ActiveRateGroup-RateGroupMedFreq,
 Module[1280]::ActiveRateGroup-RateGroupHiFreq,
 Module[1536]::CubeRoverTime,
 Module[1792]::TlmChan,
 Module[2048]::CommandDispatcher,
 Module[2304]::GroundInterface,
 Module[2560]::NetworkManager,
 Module[2816]::ActiveLogger,
 Module[3328]::Navigation,
 Module[3584]::MotorControl,
 Module[3840]::Imu,
 Module[4096]::WatchDogInterface,
 Module[4352]::Camera,
 Module[65280]::WatchdogHeartbeatTvac]

In [18]:
if magic == Magic.TELEMETRY:
    # Grab Telemetry Header:
    module_id, channel_id, data_size, VLP = VLP[:1], VLP[1:2], VLP[2:4], VLP[4:]
    print(module_id, channel_id, data_size)
    # Unpack + Format Telemetry Header:
    module_id = struct.unpack(endianness_code+'B', module_id)[0] << 8
    channel_id = struct.unpack(endianness_code+'B', channel_id)[0]
    #! TODO: Not impl. in fsw. Once is, perform check against expected size.
    # ... Rn, replace w/expected size
    data_size = struct.unpack(endianness_code+'H', data_size)[0]
    try:
        module = standards.modules[module_id]
    except KeyError:
        logger.
    exp_data_size = standards.modules[module_id] 

    print(hex(module_id), hex(channel_id), data_size)
    # data, timestamp, VLP = VLP[:data_size], VLP[data_size:data_size+4], VLP[data_size+4:]

SyntaxError: invalid syntax (<ipython-input-18-1472e03e6c7f>, line 14)

In [82]:
standards.modules['Imu'].telemetry

[((['XAcc'], 0), Channel[0]::XAcc: int16), ((['YAcc'], 1), Channel[1]::YAcc: int16), ((['ZAcc'], 2), Channel[2]::ZAcc: int16), ((['XAng'], 3), Channel[3]::XAng: uint16), ((['YAng'], 4), Channel[4]::YAng: uint16), ((['ZAng'], 5), Channel[5]::ZAng: uint16)]

In [81]:
[t.datatype for t in standards.modules['Imu'].telemetry.vals]

[<FswDataType.I16: 'int16'>,
 <FswDataType.I16: 'int16'>,
 <FswDataType.I16: 'int16'>,
 <FswDataType.U16: 'uint16'>,
 <FswDataType.U16: 'uint16'>,
 <FswDataType.U16: 'uint16'>]